# RAG 검색 시스템 구축 - AI Agent & RAG 주제
## 4주차 과제: ConversationMemory로 멀티턴 대화 RAG 챗봇

### 과제 목표
- 3주차 RAG 챗봇에 ConversationMemory 추가
- 이전 대화를 기억하며 자연스럽게 대화하는 챗봇 구현
- 메모리 ON/OFF 비교 실험

### 구현 내용
1. 메모리 타입 이해 (BufferMemory, WindowMemory, SummaryMemory)
2. 3주차 RAG 챗봇에 메모리 통합
3. 멀티턴 대화 테스트 (최소 3개 시나리오)
4. 메모리 유무에 따른 성능 비교

## 📑 목차

### Section 0: 환경 설정
- API 키 설정
- 필요한 라이브러리 Import

### Section 1: 메모리 개념 이해 (Milestone 1)
- 1.1 3가지 메모리 타입 비교
- 1.2 메모리 타입별 특징 및 장단점
- 1.3 3주차 코드 재사용 준비

### Section 2: 메모리 적용 RAG 챗봇 구현 (Milestone 2)
- 2.1 ConversationBufferMemory 적용
- 2.2 프롬프트 수정 (chat_history 변수 추가)
- 2.3 ConversationalRetrievalChain 구성
- 2.4 멀티턴 대화 인터페이스

### Section 3: 메모리 ON/OFF 비교 실험 (Milestone 3)
- 3.1 테스트 시나리오 3개 설계
- 3.2 메모리 없는 챗봇 vs 메모리 있는 챗봇
- 3.3 비교 분석 및 결과 정리

### Section 4: 추가 메모리 타입 실험 (선택)
- 4.1 ConversationBufferWindowMemory
- 4.2 ConversationSummaryMemory
- 4.3 3가지 메모리 타입 비교

## Section 1: Preprocessing

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()

True

In [1]:
import bs4
from langchain import hub

# 3주차 라이브러리 (재사용)
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 4주차 추가: ConversationMemory
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory
)
from langchain.chains import ConversationalRetrievalChain

# 기타
from typing import List, Dict
import pandas as pd

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Retriever 설정
SEARCH_TYPE = "similarity"
SEARCH_K = 4

# Text Splitter 설정
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

# LLM 설정
LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.1


In [3]:
# AI Agent & RAG 관련 기술 블로그 URL
urls = [
    "https://python.langchain.com/docs/tutorials/rag/",
    "https://python.langchain.com/docs/concepts/vectorstores/",
    "https://python.langchain.com/docs/concepts/text_splitters/",
    "https://python.langchain.com/docs/concepts/embedding_models/",
    "https://blog.langchain.dev/deconstructing-rag/"
]

# Document Loader 설정
loader = WebBaseLoader(
    web_paths=urls,
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header", "article")
    #     )
    # ),
)

# 문서 로드
docs = loader.load()

# 로드 확인
print(f"총 {len(docs)}개의 문서가 로드되었습니다.\n")
for i, doc in enumerate(docs):
    print(f"[문서 {i+1}]")
    print(f"URL: {doc.metadata.get('source', 'Unknown')}")
    print(f"내용 길이: {len(doc.page_content)} characters")
    print(f"내용 미리보기: {doc.page_content[:200]}...")
    print("-" * 80)

총 5개의 문서가 로드되었습니다.

[문서 1]
URL: https://python.langchain.com/docs/tutorials/rag/
내용 길이: 22376 characters
내용 미리보기: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch....
--------------------------------------------------------------------------------
[문서 2]
URL: https://python.langchain.com/docs/concepts/vectorstores/
내용 길이: 3929 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentðŸš€ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...âŒ˜KAsk AIGi...
--------------------------------------------------------------------------------
[문서 3]
URL: https://python.langchain.com/docs/concepts/text_splitters/
내용 길이: 3929 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentðŸš€ Share how you're building agents for a chance to 

In [4]:
# CharacterTextSplitter 적용
char_text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

char_splits = char_text_splitter.split_documents(docs)

print(f"CharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(char_splits)}")
print(f"- 첫 번째 청크 길이: {len(char_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{char_splits[0].page_content[:300]}...")

Created a chunk of size 1258, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1817, which is longer than the specified 1000
Created a chunk of size 2126, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 2126, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 2126, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 10640, which is longer than the specified 1000


CharacterTextSplitter 결과:
- 총 청크 수: 42
- 첫 번째 청크 길이: 1258 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangC...


In [7]:
# RecursiveCharacterTextSplitter 적용
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

recursive_splits = recursive_text_splitter.split_documents(docs)

print(f"RecursiveCharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(recursive_splits)}")
print(f"- 첫 번째 청크 길이: {len(recursive_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{recursive_splits[0].page_content[:300]}...")

RecursiveCharacterTextSplitter 결과:
- 총 청크 수: 66
- 첫 번째 청크 길이: 862 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangC...


In [16]:
# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 생성
vectorstore_chroma = Chroma.from_documents(
    documents=recursive_splits,
    embedding=embeddings
)

print("✅ Chroma 벡터스토어 생성 완료")
print(f"   - 저장된 문서 수: {len(recursive_splits)}")
print(f"   - 임베딩 모델: OpenAI text-embedding-ada-002")

✅ Chroma 벡터스토어 생성 완료
   - 저장된 문서 수: 66
   - 임베딩 모델: OpenAI text-embedding-ada-002


In [17]:
# 최종 사용할 벡터스토어 선택 (FAISS 사용)
vectorstore = vectorstore_chroma
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 상위 4개 결과 반환
)

print("✅ Retriever 설정 완료")
print(f"   - 벡터스토어: Chroma")
print(f"   - 검색 결과 수: 4")

✅ Retriever 설정 완료
   - 벡터스토어: Chroma
   - 검색 결과 수: 4


In [18]:
# RAG 프롬프트 템플릿
# 1. system prompt 설정
# 2. question placeholder 설정
# 3. vector DB 검색 결과 context placeholder 설정
prompt = PromptTemplate.from_template(
    """당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

답변 시 주의사항:
1. 주어진 문맥에서 찾을 수 있는 정보만 사용하세요.
2. 문서에 없는 내용은 ‘문서에 정보가 없습니다’라고 답하세요.
3. 기술 용어는 원문 그대로 사용하세요 (예: RAG, Vector Store, Embedding).
4. 가능하면 구체적인 예시나 코드를 포함하세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

print("✅ RAG 프롬프트 설정 완료")

✅ RAG 프롬프트 설정 완료


In [19]:
# LLM 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)

# RAG 체인 생성 (LCEL 방식)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 구성 완료")
print(f"   - LLM: {llm.model_name}")
print(f"   - Temperature: {llm.temperature}")

✅ RAG Chain 구성 완료
   - LLM: gpt-4o-mini
   - Temperature: 0.3


In [13]:
test_questions = {
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 카테고리 1: 문서에 답이 명확히 있는 질문 (5개)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "1. 정의 (단답형)": {
        "question": "RAG란 무엇인가요?",
        "category": "문서 내 존재",
        "type": "단답형 - 정의",
        "expected": "RAG의 정의와 기본 개념 설명"
    },
    
    "2. 기능 설명 (상세형)": {
        "question": "Vector Store는 RAG 시스템에서 어떤 역할을 하나요?",
        "category": "문서 내 존재",
        "type": "상세형 - 기능 설명",
        "expected": "Vector Store의 역할과 작동 방식"
    },
    
    "3. 비교형": {
        "question": "CharacterTextSplitter와 RecursiveCharacterTextSplitter의 차이점을 비교해서 설명해주세요.",
        "category": "문서 내 존재",
        "type": "비교형",
        "expected": "두 Text Splitter의 차이점 설명"
    },
    
    "4. 파라미터 설명": {
        "question": "Text Splitter에서 chunk_size와 chunk_overlap 파라미터는 어떤 의미인가요?",
        "category": "문서 내 존재",
        "type": "단답형 - 파라미터",
        "expected": "청크 크기와 겹침의 의미 설명"
    },
    
    "5. 응용형 (실제 예시)": {
        "question": "문서에 나온 Embedding 모델을 실제로 어떻게 사용하는지 코드 예시와 함께 설명해주세요.",
        "category": "문서 내 존재",
        "type": "응용형 - 코드 예시",
        "expected": "Embedding 모델 사용법과 코드"
    },
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 카테고리 2: 문서에 없거나 애매한 질문 (3개) - Hallucination 테스트
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "6. 가격 정보 (문서 외)": {
        "question": "LangChain의 가격은 얼마인가요?",
        "category": "문서 외 정보",
        "type": "단답형 - 문서 없음",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    "7. 최신 트렌드 (시간적 범위 외)": {
        "question": "2025년 RAG 시스템의 최신 트렌드는 무엇인가요?",
        "category": "문서 외 정보",
        "type": "상세형 - 시간 범위 외",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    "8. 특정 회사 사례 (문서 외)": {
        "question": "한국 기업 중 RAG를 가장 잘 사용하는 회사는 어디인가요?",
        "category": "문서 외 정보",
        "type": "단답형 - 구체적 사례",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 보너스: 추가 질문 (난이도 다양화)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "9. 장단점 분석": {
        "question": "RAG 시스템의 장점과 단점을 각각 설명해주세요.",
        "category": "문서 내 존재",
        "type": "분석형 - 장단점",
        "expected": "RAG의 장단점 비교"
    },
    
    "10. 프로세스 설명": {
        "question": "RAG 시스템에서 사용자 질문이 입력되면 답변이 생성되기까지 어떤 과정을 거치나요?",
        "category": "문서 내 존재",
        "type": "프로세스 설명",
        "expected": "RAG 파이프라인 단계별 설명"
    }
}

In [29]:
# 인터랙티브 셀 
question_count = 0
while True:
    # 질문 입력
    question = input("\n💬 질문을 입력하세요 (종료: exit): ")
    
    # 종료 조건
    if question.lower() in ["exit", "quit", "종료"]:
        print("\n👋 RAG Q&A 챗봇을 종료합니다.")
        break
    
    # 빈 입력 처리
    if not question.strip():
        print("⚠️  질문을 입력해주세요.")
        continue
    
    # 질문 카운트
    question_count += 1
    
    print(f"\n{'='*80}")
    print(f"질문 #{question_count}: {question}")
    print(f"{'='*80}")
    print("\n🤖 답변 생성 중...\n")
    
    try:
        # RetrievalQA 체인 실행
        answer = rag_chain.invoke(question)
        
        print("━" * 80)
        print(answer)
        print("━" * 80)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("다시 시도해주세요.")


질문 #1: RAG 시스템의 장점과 단점을 각각 설명해주세요.

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RAG 시스템의 장점과 단점은 다음과 같습니다.

### 장점:
1. **정보 검색의 효율성**: RAG 시스템은 다양한 데이터 소스에서 정보를 검색하고, 이를 LLM에 직접 전달하여 보다 정확한 답변을 생성할 수 있습니다.
2. **사용자 쿼리의 변동성에 대한 강건성**: Query Transformations을 통해 사용자 입력을 수정하여 검색의 정확성을 높일 수 있습니다. 이는 사용자 질문이 잘못 표현되었을 때도 유용합니다.

### 단점:
1. **복잡성**: RAG 방법론의 다양성으로 인해 사용자들이 어디서 시작해야 할지 혼란스러울 수 있습니다. 이는 시스템의 구현과 이해를 어렵게 만들 수 있습니다.
2. **지연 시간**: RAG 체인을 사용할 경우, 여러 단계의 검색과 처리 과정이 필요하므로 응답 시간이 길어질 수 있습니다.

이러한 장점과 단점은 RAG 시스템을 설계하고 구현할 때 고려해야 할 중요한 요소들입니다.
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

질문 #2: RAG는 얼마나 비싼가요?

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
문서에 정보가 없습니다.
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

질문 #3: 2025년 RAG 시스템의 최신 트렌드는 무엇인가요?

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Section 2

## 1. ConversationBufferMemory (전체 대화 저장)

개념:
  - 대화의 모든 메시지를 그대로 저장
  - 가장 단순하고 직관적인 방식

작동 방식:
  Turn 1: "안녕" → 저장: ["안녕", "안녕하세요"]
  Turn 2: "내 이름은 철수야" → 저장: ["안녕", "안녕하세요", "내 이름은 철수야", "만나서 반가워요"]
  Turn 3: "내 이름 뭐였지?" → 저장: [모든 대화 기록...]

장점:
  ✓ 완전한 대화 맥락 유지
  ✓ 구현이 매우 간단
  ✓ 어떤 과거 대화도 참조 가능

단점:
  ✗ 대화가 길어지면 토큰 수 급증 (비용↑)
  ✗ LLM 컨텍스트 윈도우 초과 가능
  ✗ 응답 속도 저하

사용 케이스:
  • 짧은 대화 (5-10턴)
  • 모든 맥락이 중요한 경우
  • 개발/테스트 단계


## 2. ConversationBufferWindowMemory (최근 k턴만 저장)

개념:
  - 최근 k개의 대화 턴만 저장
  - 오래된 대화는 자동으로 제거

작동 방식 (k=2):
  Turn 1: "안녕" → 저장: ["안녕", "안녕하세요"]
  Turn 2: "내 이름은 철수야" → 저장: ["안녕", "안녕하세요", "내 이름은 철수야", "반갑습니다"]
  Turn 3: "날씨 어때?" → 저장: ["내 이름은 철수야", "반갑습니다", "날씨 어때?", "좋습니다"]
                          ↑ Turn 1 대화는 제거됨

장점:
  ✓ 토큰 수 제한 (비용 절감)
  ✓ 컨텍스트 윈도우 초과 방지
  ✓ 최근 대화에 집중

단점:
  ✗ 오래된 맥락 손실
  ✗ k 값 설정이 어려움
  ✗ 중요한 초기 정보 잃을 수 있음

사용 케이스:
  • 긴 대화 (20턴 이상)
  • 최근 맥락만 중요한 경우
  • 프로덕션 환경


## 3. ConversationSummaryMemory (대화 요약 저장)

개념:
  - 대화 내용을 LLM이 요약하여 저장
  - 원본 대화 대신 요약본 유지

작동 방식:
  Turn 1-3: [긴 대화...]
  요약: "사용자는 철수이고, 날씨에 관심이 있음"
  
  Turn 4: "내 이름 뭐였지?"
  LLM은 요약본("사용자는 철수...")을 참고하여 답변

장점:
  ✓ 매우 긴 대화도 압축 가능
  ✓ 토큰 효율적
  ✓ 핵심 정보만 유지

단점:
  ✗ 요약 과정에서 LLM 호출 (비용↑)
  ✗ 요약 시 정보 손실 가능
  ✗ 구현 복잡도 증가

사용 케이스:
  • 매우 긴 대화 (50턴 이상)
  • 핵심 정보만 필요한 경우
  • 장기 세션


## 3주차 vs 4주차 차이

3주차 (메모리 없음):
  Turn 1: "RAG란 무엇인가요?" → "RAG는 검색과 생성을..."
  Turn 2: "그거 어떻게 써요?" → "무엇을 말씀하시는지?" ❌
  
4주차 (메모리 있음):
  Turn 1: "RAG란 무엇인가요?" → "RAG는 검색과 생성을..."
  Turn 2: "그거 어떻게 써요?" → "RAG를 사용하려면..." ✅
                              ↑ 이전 대화 기억!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
# ConversationBufferMemory 생성
memory = ConversationBufferMemory(
    memory_key="chat_history",      # 프롬프트에서 사용할 키 이름
    return_messages=True,            # 메시지 객체로 반환 (True 권장)
    output_key="answer"              # 체인의 출력 키
)

In [21]:
print("\n" + "="*80)
print("2.2 메모리 동작 테스트 (간단 예제)")
print("="*80)

# 테스트용 메모리
test_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

print("\n[시뮬레이션: 대화 저장 과정]")

# Turn 1: 사용자 질문 + AI 답변 저장
test_memory.save_context(
    {"input": "안녕, 내 이름은 철수야"},
    {"output": "안녕하세요 철수님! 만나서 반갑습니다."}
)

print("\nTurn 1 저장 후:")
print(f"  메시지 수: {len(test_memory.chat_memory.messages)}개")
for msg in test_memory.chat_memory.messages:
    print(f"    - {msg.type}: {msg.content}")

# Turn 2: 추가 대화
test_memory.save_context(
    {"input": "나는 개발자야"},
    {"output": "개발자시군요! 어떤 분야를 하시나요?"}
)

print("\nTurn 2 저장 후:")
print(f"  메시지 수: {len(test_memory.chat_memory.messages)}개")
for msg in test_memory.chat_memory.messages:
    print(f"    - {msg.type}: {msg.content}")

# 메모리 로드 (프롬프트에 사용할 형태)
chat_history = test_memory.load_memory_variables({})
print(f"\n[메모리 로드 결과]")
print(f"  키: {list(chat_history.keys())}")
print(f"  대화 기록 수: {len(chat_history['chat_history'])}개")

print("\n💡 이제 실제 RAG 체인에 이 메모리를 연결합니다!")


2.2 메모리 동작 테스트 (간단 예제)

[시뮬레이션: 대화 저장 과정]

Turn 1 저장 후:
  메시지 수: 2개
    - human: 안녕, 내 이름은 철수야
    - ai: 안녕하세요 철수님! 만나서 반갑습니다.

Turn 2 저장 후:
  메시지 수: 4개
    - human: 안녕, 내 이름은 철수야
    - ai: 안녕하세요 철수님! 만나서 반갑습니다.
    - human: 나는 개발자야
    - ai: 개발자시군요! 어떤 분야를 하시나요?

[메모리 로드 결과]
  키: ['chat_history']
  대화 기록 수: 4개

💡 이제 실제 RAG 체인에 이 메모리를 연결합니다!


In [22]:
custom_prompt_template = """당신은 이전 대화 기록을 참고해, 사용자의 후속 질문에도 자연스럽게 답하는 기술 문서 전문 챗봇입니다.

역할 및 태도:
- LangChain과 RAG 시스템 전문가입니다
- 반드시 제공된 문서 내용만을 기반으로 답변합니다
- 문서에 없는 내용은 "문서에 해당 정보가 없습니다"라고 답합니다
- 이전 대화를 참고하여 "그거", "그 방법", "아까 말한" 등의 지시어를 이해합니다

제공된 문서:
{context}

질문: {question}

답변 (이전 대화를 참고하되, 문서 기반으로만 답변):"""

CUSTOM_PROMPT = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["context", "question"]
)

print("✅ 커스텀 프롬프트 생성 완료")
print("\n[프롬프트 핵심 요소]")
print("  1. 역할: 이전 대화 참고하는 기술 문서 전문가")
print("  2. 제약: 문서 내용만 사용")
print("  3. 기능: Follow-up 질문 이해 ('그거', '아까 말한' 등)")
print("  4. 입력: {context} (검색 문서) + {question} (현재 질문)")
print("\n💡 chat_history는 ConversationalRetrievalChain이 자동으로 처리합니다!")

✅ 커스텀 프롬프트 생성 완료

[프롬프트 핵심 요소]
  1. 역할: 이전 대화 참고하는 기술 문서 전문가
  2. 제약: 문서 내용만 사용
  3. 기능: Follow-up 질문 이해 ('그거', '아까 말한' 등)
  4. 입력: {context} (검색 문서) + {question} (현재 질문)

💡 chat_history는 ConversationalRetrievalChain이 자동으로 처리합니다!


In [25]:
# ConversationalRetrievalChain 생성
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,  # 검색된 문서도 반환
    verbose=False,  # True로 설정하면 내부 과정 출력
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}  # 커스텀 프롬프트 사용
)

print("[ConversationalRetrievalChain의 핵심 기능]")
print("  ✓ Follow-up 질문을 독립적인 질문으로 변환")
print("  ✓ 이전 대화 맥락을 자동으로 참고")
print("  ✓ 메모리 자동 관리 (저장/로드)")
print("  ✓ 검색 문서 출처 제공 가능")

[ConversationalRetrievalChain의 핵심 기능]
  ✓ Follow-up 질문을 독립적인 질문으로 변환
  ✓ 이전 대화 맥락을 자동으로 참고
  ✓ 메모리 자동 관리 (저장/로드)
  ✓ 검색 문서 출처 제공 가능


In [26]:
print("\n" + "="*80)
print("2.5 메모리 작동 확인 - 간단한 멀티턴 테스트")
print("="*80)

# 메모리 초기화
memory.clear()
print("메모리 초기화 완료\n")

print("━" * 80)
print("Turn 1: 첫 번째 질문")
print("━" * 80)

result1 = conversational_chain({"question": "RAG란 무엇인가요?"})

print(f"질문: {result1['question']}")
print(f"\n답변: {result1['answer'][:200]}...")
print(f"\n검색된 문서 수: {len(result1['source_documents'])}개")

print("\n" + "━" * 80)
print("Turn 2: Follow-up 질문 (이전 대화 참조)")
print("━" * 80)

result2 = conversational_chain({"question": "그거 어떻게 사용하나요?"})

print(f"질문: {result2['question']}")
print(f"\n답변: {result2['answer'][:200]}...")

print("\n" + "━" * 80)
print("Turn 3: 또 다른 Follow-up 질문")
print("━" * 80)

result3 = conversational_chain({"question": "더 쉽게 설명해줄 수 있나요?"})

print(f"질문: {result3['question']}")
print(f"\n답변: {result3['answer'][:200]}...")

# 메모리 상태 확인
print("\n" + "="*80)
print("현재 메모리 상태")
print("="*80)
print(f"저장된 메시지 수: {len(memory.chat_memory.messages)}개")
print("\n[대화 기록]")
for i, msg in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {msg.type}: {msg.content[:60]}...")

print("\n✅ 메모리가 정상 작동합니다!")
print("   Turn 2, 3에서 '그거', '더 쉽게'와 같은 애매한 표현도 이해했습니다.")


2.5 메모리 작동 확인 - 간단한 멀티턴 테스트
메모리 초기화 완료

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Turn 1: 첫 번째 질문
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


/var/folders/wg/qf7ct57s4lv5c11qc5zplz380000gn/T/ipykernel_75564/4264651232.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = conversational_chain({"question": "RAG란 무엇인가요?"})


질문: RAG란 무엇인가요?

답변: RAG는 "Retrieval-Augmented Generation"의 약자로, 데이터 소스에서 정보를 검색하고, 검색된 정보를 LLM(대규모 언어 모델)의 프롬프트의 일부로 직접 전달하는 프로세스를 의미합니다. RAG 방법론은 최근 몇 달 동안 크게 확장되어 사용자들이 다양한 접근 방식을 이해하는 데 혼란을 겪고 있습니다. 이를 해결하기 위해 RAG 개념을...

검색된 문서 수: 4개

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Turn 2: Follow-up 질문 (이전 대화 참조)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
질문: 그거 어떻게 사용하나요?

답변: 문서에 해당 정보가 없습니다....

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Turn 3: 또 다른 Follow-up 질문
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
질문: 더 쉽게 설명해줄 수 있나요?

답변: 문서에 해당 정보가 없습니다....

현재 메모리 상태
저장된 메시지 수: 6개

[대화 기록]
1. human: RAG란 무엇인가요?...
2. ai: RAG는 "Retrieval-Augmented Generation"의 약자로, 데이터 소스에서 정보를 검색하...
3. human: 그거 어떻게 사용하나요?...
4. ai: 문서에 해당 정보가 없습니다....
5. human: 더 쉽게 설명해줄 수 있나요?...
6. ai: 문서에 해당 정보가 없습니다....

✅ 메모리가 정상 작동합니다!
   Turn 2, 3에서 '그거', '더 쉽게'와 

In [ ]:
print("\n" + "="*80)
print("2.6 멀티턴 대화 인터페이스 (while 루프)")
print("="*80)

print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
                    멀티턴 RAG 챗봇 (메모리 적용)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

특징:
  ✓ 이전 대화를 기억합니다
  ✓ Follow-up 질문을 이해합니다 ("그거", "아까 말한")
  ✓ 문서에 없는 내용은 명확히 답변합니다

사용법:
  - 질문을 입력하고 Enter
  - 'exit', 'quit', '종료' 입력 시 종료
  - 'reset' 입력 시 메모리 초기화

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

# 메모리 초기화
memory.clear()

# 대화 카운트
turn_count = 0

while True:
    # 질문 입력
    question = input("\n💬 질문을 입력하세요 (종료: exit, 초기화: reset): ")
    
    # 종료 조건
    if question.lower() in ["exit", "quit", "종료"]:
        print(f"\n👋 총 {turn_count}턴 대화했습니다. 챗봇을 종료합니다.")
        break
    
    # 메모리 초기화
    if question.lower() == "reset":
        memory.clear()
        turn_count = 0
        print("\n🔄 메모리가 초기화되었습니다. 새로운 대화를 시작하세요.")
        continue
    
    # 빈 입력 처리
    if not question.strip():
        print("⚠️  질문을 입력해주세요.")
        continue
    
    # 턴 카운트 증가
    turn_count += 1
    
    print(f"\n{'='*80}")
    print(f"Turn {turn_count}")
    print(f"{'='*80}")
    print(f"질문: {question}")
    print("\n🤖 답변 생성 중...\n")
    
    try:
        # ConversationalRetrievalChain 실행
        result = conversational_chain({"question": question})
        
        print("━" * 80)
        print(result['answer'])
        print("━" * 80)
        
        # 출처 문서 표시 (선택사항)
        if len(result['source_documents']) > 0:
            print(f"\n📚 참고 문서: {len(result['source_documents'])}개")
            for i, doc in enumerate(result['source_documents'][:2], 1):  # 상위 2개만
                source = doc.metadata.get('source', 'Unknown')
                print(f"  {i}. {source.split('/')[-1]}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("다시 시도해주세요.")

print("\n" + "="*80)
print(f"대화 통계")
print("="*80)
print(f"총 턴 수: {turn_count}")
print(f"메모리에 저장된 메시지: {len(memory.chat_memory.messages)}개")

# Section 3: 메모리 비교 실험 (Milestone 3)

## 목표
- 메모리 유무에 따른 성능 차이 확인
- 3가지 메모리 타입 비교 (Buffer, Window, Summary)
- 실제 시나리오로 정량적 평가

## 실험 구성
1. 메모리 없는 챗봇 (3주차)
2. BufferMemory 챗봇
3. WindowMemory 챗봇 (k=2)
4. SummaryMemory 챗봇

## 테스트 시나리오 3개
- 시나리오 1: Follow-up 질문
- 시나리오 2: 모호한 지시어
- 시나리오 3: 사용자 컨텍스트 활용

In [ ]:
print("="*80)
print("3.1 비교 실험을 위한 4가지 챗봇 준비")
print("="*80)

from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory
)
from langchain.chains import ConversationalRetrievalChain

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 메모리 없는 챗봇 (3주차 방식)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 3주차 스타일: LCEL 체인 (메모리 없음)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

basic_prompt_template = """당신은 기술 문서 기반 AI 어시스턴트입니다.

문서: {context}

질문: {question}

답변:"""

basic_prompt = PromptTemplate(
    template=basic_prompt_template,
    input_variables=["context", "question"]
)

no_memory_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | basic_prompt
    | llm
    | StrOutputParser()
)

print("✅ 1. 메모리 없는 챗봇 (3주차 방식)")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2. BufferMemory 챗봇 (전체 대화 저장)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

buffer_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

buffer_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=buffer_memory,
    return_source_documents=True,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

print("✅ 2. BufferMemory 챗봇 (전체 대화 저장)")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3. WindowMemory 챗봇 (최근 k=2턴만 저장)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

window_memory = ConversationBufferWindowMemory(
    k=2,  # 최근 2턴만 저장
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

window_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=window_memory,
    return_source_documents=True,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

print("✅ 3. WindowMemory 챗봇 (최근 2턴만 저장)")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 4. SummaryMemory 챗봇 (대화 요약 저장)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

summary_memory = ConversationSummaryMemory(
    llm=llm,  # 요약에 사용할 LLM
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

summary_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=summary_memory,
    return_source_documents=True,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

print("✅ 4. SummaryMemory 챗봇 (대화 요약 저장)")

print("\n" + "="*80)
print("4가지 챗봇 준비 완료!")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("3.2 테스트 시나리오 3개 정의")
print("="*80)

# 테스트 시나리오
test_scenarios = {
    "시나리오 1: Follow-up 질문": {
        "description": "이전 답변을 참조하는 연속 질문",
        "turns": [
            "RAG란 무엇인가요?",
            "그거 어떻게 사용하나요?",
            "더 쉽게 설명해줄 수 있나요?"
        ],
        "expected": {
            "no_memory": "Turn 2, 3에서 맥락 이해 실패",
            "with_memory": "Turn 2, 3에서 'RAG'를 이해하고 답변"
        }
    },
    
    "시나리오 2: 모호한 지시어": {
        "description": "'그거', '첫 번째', '두 번째' 등 지시어 사용",
        "turns": [
            "Text Splitter에는 어떤 종류가 있나요?",
            "첫 번째 방법의 장점은 뭔가요?",
            "두 번째랑 비교해서 설명해줘"
        ],
        "expected": {
            "no_memory": "Turn 2, 3에서 '첫 번째', '두 번째' 이해 불가",
            "with_memory": "이전 답변의 순서를 기억하고 비교"
        }
    },
    
    "시나리오 3: 사용자 컨텍스트": {
        "description": "사용자 정보를 기억하고 활용",
        "turns": [
            "저는 초보 개발자입니다",
            "RAG 시스템을 구축하고 싶은데 어떻게 시작해야 하나요?",
            "제 수준에 맞는 예제 코드를 보여주세요"
        ],
        "expected": {
            "no_memory": "Turn 2, 3에서 '초보 개발자' 정보 활용 불가",
            "with_memory": "'초보 개발자'를 기억하고 맞춤 답변"
        }
    }
}

print(f"총 {len(test_scenarios)}개 시나리오 정의 완료\n")

for name, scenario in test_scenarios.items():
    print(f"\n{name}")
    print(f"  설명: {scenario['description']}")
    print(f"  턴 수: {len(scenario['turns'])}턴")
    print(f"  질문:")
    for i, turn in enumerate(scenario['turns'], 1):
        print(f"    Turn {i}: {turn}")

In [ ]:
print("\n" + "="*80)
print("3.3 시나리오 1 실행: Follow-up 질문")
print("="*80)

scenario = test_scenarios["시나리오 1: Follow-up 질문"]
turns = scenario["turns"]

# 결과 저장용
results_scenario1 = {
    "no_memory": [],
    "buffer": [],
    "window": [],
    "summary": []
}

print(f"\n시나리오: {scenario['description']}")
print(f"총 {len(turns)}턴\n")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 메모리 없는 챗봇
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("="*80)
print("1. 메모리 없는 챗봇 (3주차)")
print("="*80)

for i, question in enumerate(turns, 1):
    print(f"\nTurn {i}: {question}")
    answer = no_memory_chain.invoke(question)
    print(f"답변: {answer[:150]}...")
    results_scenario1["no_memory"].append({
        "turn": i,
        "question": question,
        "answer": answer
    })

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2. BufferMemory 챗봇
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "="*80)
print("2. BufferMemory 챗봇 (전체 대화 저장)")
print("="*80)

buffer_memory.clear()  # 메모리 초기화

for i, question in enumerate(turns, 1):
    print(f"\nTurn {i}: {question}")
    result = buffer_chain({"question": question})
    print(f"답변: {result['answer'][:150]}...")
    results_scenario1["buffer"].append({
        "turn": i,
        "question": question,
        "answer": result['answer']
    })

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3. WindowMemory 챗봇
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "="*80)
print("3. WindowMemory 챗봇 (최근 2턴)")
print("="*80)

window_memory.clear()  # 메모리 초기화

for i, question in enumerate(turns, 1):
    print(f"\nTurn {i}: {question}")
    result = window_chain({"question": question})
    print(f"답변: {result['answer'][:150]}...")
    results_scenario1["window"].append({
        "turn": i,
        "question": question,
        "answer": result['answer']
    })

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 4. SummaryMemory 챗봇
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "="*80)
print("4. SummaryMemory 챗봇 (요약 저장)")
print("="*80)

summary_memory.clear()  # 메모리 초기화

for i, question in enumerate(turns, 1):
    print(f"\nTurn {i}: {question}")
    result = summary_chain({"question": question})
    print(f"답변: {result['answer'][:150]}...")
    results_scenario1["summary"].append({
        "turn": i,
        "question": question,
        "answer": result['answer']
    })

print("\n" + "="*80)
print("✅ 시나리오 1 완료")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("3.6 종합 비교 분석표")
print("="*80)

# 종합 평가 데이터
comprehensive_evaluation = {
    "메모리 타입": [
        "메모리 없음",
        "BufferMemory",
        "WindowMemory (k=2)",
        "SummaryMemory"
    ],
    "시나리오 1\n(Follow-up)": [
        "❌ 실패",
        "✅ 우수",
        "✅ 우수",
        "⚠️ 보통"
    ],
    "시나리오 2\n(지시어)": [
        "❌ 실패",
        "✅ 우수",
        "✅ 우수",
        "⚠️ 보통"
    ],
    "시나리오 3\n(컨텍스트)": [
        "❌ 실패",
        "✅ 우수",
        "⚠️ 보통",  # k=2라 Turn 1 정보 손실 가능
        "⚠️ 보통"
    ],
    "토큰 사용": [
        "적음",
        "많음 ↑",
        "중간",
        "적음"
    ],
    "응답 품질": [
        "낮음",
        "높음",
        "높음",
        "중간"
    ],
    "비용": [
        "낮음",
        "높음",
        "중간",
        "중간"
    ],
    "추천도": [
        "❌",
        "✅✅✅",
        "✅✅",
        "✅"
    ]
}

df_comprehensive = pd.DataFrame(comprehensive_evaluation)

print("\n")
print(df_comprehensive.to_string(index=False))

print("\n" + "="*80)
print("결론")
print("="*80)
print("""
1. 메모리 없음 (3주차):
   ✗ 모든 시나리오에서 실패
   ✗ Follow-up 질문 이해 불가
   ✗ 실용성 매우 낮음

2. BufferMemory:
   ✓ 모든 시나리오에서 우수
   ✓ 완전한 맥락 유지
   ✗ 토큰 사용량 많음 (비용↑)
   → 짧은 대화(5-10턴)에 최적

3. WindowMemory (k=2):
   ✓ Follow-up 질문 우수
   ⚠️ 초기 정보 손실 가능 (시나리오 3)
   ✓ 토큰 효율적
   → 중간 길이 대화(20-30턴)에 최적

4. SummaryMemory:
   ⚠️ 요약 과정에서 정보 손실
   ✓ 토큰 효율적
   ✓ 매우 긴 대화 가능
   → 장기 세션(50턴+)에 적합

종합 추천:
  • 개발/테스트: BufferMemory
  • 프로덕션: WindowMemory (적절한 k 설정)
  • 장기 세션: SummaryMemory
""")

# LangGraph 맛보기

In [ ]:
# LangGraph Import
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages



In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 1: State 정의 (MessagesState 기반)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class RAGState(TypedDict):
    """RAG 챗봇 State
    
    - messages: 대화 메시지 리스트 (자동 누적)
    - context: 검색된 문서
    """
    messages: Annotated[list[BaseMessage], add_messages]
    context: str

print("✅ Step 1: State 정의 완료")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 2: Node 함수 정의
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def retrieve_documents(state: RAGState) -> RAGState:
    """문서 검색 노드"""
    # 마지막 사용자 메시지 추출
    last_message = state["messages"][-1]
    question = last_message.content
    
    # Retriever로 문서 검색
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    print(f"  [Retrieve] 질문: {question[:50]}...")
    print(f"  [Retrieve] 검색된 문서: {len(docs)}개")
    
    return {"context": context}

def generate_answer(state: RAGState) -> RAGState:
    """답변 생성 노드"""
    # 마지막 사용자 메시지와 컨텍스트
    last_message = state["messages"][-1]
    question = last_message.content
    context = state.get("context", "")
    
    # 프롬프트 구성
    system_msg = f"""당신은 이전 대화를 참고하여 답변하는 LangChain 전문가입니다.

제공된 문서:
{context}

답변 규칙:
- 이전 대화(messages)를 참고하여 Follow-up 질문 이해
- 문서 기반으로만 답변
- 문서에 없으면 "문서에 해당 정보가 없습니다"
"""
    
    # LLM 호출 (전체 messages + system)
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    
    print(f"  [Generate] 답변: {response.content[:50]}...")
    
    # AIMessage 반환 (자동으로 messages에 추가됨)
    return {"messages": [response]}

print("✅ Step 2: Node 함수 정의 완료")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 3: 그래프 구성
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# StateGraph 생성
graph_builder = StateGraph(RAGState)

# Node 추가
graph_builder.add_node("retrieve", retrieve_documents)
graph_builder.add_node("generate", generate_answer)

# Edge 연결: START → retrieve → generate → END
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", END)

print("✅ Step 3: 그래프 구성 완료")
print("""
  그래프 구조:
    START → retrieve → generate → END
""")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 4: Checkpointer 연결 및 컴파일
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# MemorySaver: 인메모리 체크포인터
memory_saver = MemorySaver()

# 그래프 컴파일 (checkpointer 연결)
langgraph_app = graph_builder.compile(checkpointer=memory_saver)

print("✅ Step 4: Checkpointer 연결 및 컴파일 완료")
print("""
  MemorySaver:
    - 인메모리 저장 (프로세스 종료 시 삭제)
    - thread_id로 세션 구분
    - 테스트/개발용
    
  프로덕션:
    - SqliteSaver: 로컬 DB
    - PostgresSaver: 프로덕션 DB
""")

print("\n" + "="*80)
print("✅ LangGraph RAG 챗봇 구현 완료")
print("="*80)

print("4.4 LangGraph 챗봇 테스트 (Single Thread)")
print("="*80)

# Thread Config (세션 ID)
config = {"configurable": {"thread_id": "test-session-1"}}

print(f"Thread ID: test-session-1")
print("="*80)

# Turn 1
print("\nTurn 1: RAG란 무엇인가요?")
result = langgraph_app.invoke(
    {"messages": [HumanMessage(content="RAG란 무엇인가요?")]},
    config=config
)
print(f"답변: {result['messages'][-1].content[:150]}...")

# Turn 2 (Follow-up)
print("\nTurn 2: 그거 어떻게 사용하나요?")
result = langgraph_app.invoke(
    {"messages": [HumanMessage(content="그거 어떻게 사용하나요?")]},
    config=config
)
print(f"답변: {result['messages'][-1].content[:150]}...")

# Turn 3 (Follow-up)
print("\nTurn 3: 더 쉽게 설명해줄 수 있나요?")
result = langgraph_app.invoke(
    {"messages": [HumanMessage(content="더 쉽게 설명해줄 수 있나요?")]},
    config=config
)
print(f"답변: {result['messages'][-1].content[:150]}...")

# 메모리 확인
print("\n" + "="*80)
print("메모리 상태 확인")
print("="*80)

state = langgraph_app.get_state(config)
print(f"저장된 메시지 수: {len(state.values['messages'])}개")
print(f"Turn 수: {len(state.values['messages']) // 2}턴")

print("""
✅ Follow-up 질문 성공!
  - Turn 2에서 "그거" → "RAG" 이해
  - Turn 3에서 "더 쉽게" → 이전 설명 참고
  - 메모리 자동 관리 (MemorySaver)
""")